# Healthcare Billing Audit Analysis

## Problem Statement

In the healthcare industry, particularly within the pharmacy sector, accurate and efficient billing processes are crucial for financial stability and regulatory compliance. However, billing data often contains errors, inconsistencies, and patterns that, if left unaddressed, can lead to revenue loss, compliance issues, and decreased patient satisfaction.

This project aims to develop a comprehensive workflow to audit billing data in the healthcare/pharmacy sector. By leveraging data analysis techniques and visualization tools, we seek to ensure data accuracy, identify discrepancies, analyze billing patterns, and generate actionable insights to improve the overall billing process.


## Task

1. Data Collection:
   - Load the provided sample dataset or a synthetic dataset
   - Clean and standardize the data

2. Data Validation:
   - Validate data accuracy and completeness
   - Check for missing, incorrect, or inconsistent data entries

3. Billing Analysis:
   - Analyze billing data for accuracy and timeliness
   - Identify discrepancies or unusual patterns
   - Analyze metrics such as:
     - Billing cycle times
     - Payment discrepancies
     - Late payment occurrences
     - Average payment amounts

4. Report Generation:
   - Generate a detailed billing report
   - Use a Large Language Model (LLM) for analysis and report generation
   - Include insights and findings from the billing analysis
   - Summarize overall accuracy and efficiency of the billing process
   - Automate this step as part of the pipeline

 ****Data Loading****

In [1]:
import pandas as pd

def load_data(file_path):
    df = pd.read_excel(file_path, engine='openpyxl')
    print("Data loaded successfully. Shape:", df.shape)
    print("\nFirst few rows:")
    print(df.head())
    print("\nColumn names:", df.columns.tolist())
    print("\nData types:")
    print(df.dtypes)
    return df

# Load the data
df = load_data(r'C:\Users\gaganguru\Dataset.xlsx')

Data loaded successfully. Shape: (110, 7)

First few rows:
   Account ID  Customer Name Billing Date      Amount   Status   Description  \
0        1001   David Wilson   2023-01-01  193.987558  Pending  Consultation   
1        1002  Alice Johnson   2023-01-02  370.656021  Pending      Purchase   
2        1003  Sophia Garcia   2023-01-03  233.019486  Pending      Lab Test   
3        1004   Olivia Davis   2023-01-04  296.120725     Paid     Follow-up   
4        1005   David Wilson   2023-01-05  294.654764     Paid       Therapy   

    Department  
0   Pediatrics  
1  Orthopedics  
2     Pharmacy  
3     Pharmacy  
4          ENT  

Column names: ['Account ID', 'Customer Name', 'Billing Date', 'Amount', 'Status', 'Description', 'Department']

Data types:
Account ID                int64
Customer Name            object
Billing Date     datetime64[ns]
Amount                  float64
Status                   object
Description              object
Department               object
dtype: ob

****Data Validation****

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

def validate_data(df):
    print("Data Validation Report")
    print("=====================")

    # Check for missing values
    missing_values = df.isnull().sum()
    print("\n1. Missing Values:")
    print(missing_values[missing_values > 0])

    # Check for duplicate transactions
    duplicates = df[df.duplicated()]
    print(f"\n2. Duplicate Transactions: {len(duplicates)}")

    # Check for non-standard statuses
    status_column = 'Status' if 'Status' in df.columns else 'status' if 'status' in df.columns else None
    if status_column:
        valid_statuses = ['Paid', 'Unpaid', 'Pending']
        invalid_statuses = df[~df[status_column].isin(valid_statuses)]
        print(f"\n3. Non-standard Statuses: {len(invalid_statuses)}")
        if len(invalid_statuses) > 0:
            print("Unique non-standard statuses:", df[status_column].unique().tolist())
    else:
        print("\n3. Status column not found. Skipping status check.")

    # Check for negative amounts
    amount_column = 'Amount' if 'Amount' in df.columns else 'amount' if 'amount' in df.columns else None
    if amount_column:
        negative_amounts = df[df[amount_column] < 0]
        print(f"\n4. Negative Amounts: {len(negative_amounts)}")
    else:
        print("\n4. Amount column not found. Skipping negative amount check.")

    # Check for future billing dates
    billing_date_column = 'Billing Date' if 'Billing Date' in df.columns else 'billing_date' if 'billing_date' in df.columns else None
    if billing_date_column:
        df[billing_date_column] = pd.to_datetime(df[billing_date_column], errors='coerce')
        future_dates = df[df[billing_date_column] > datetime.now()]
        print(f"\n5. Future Billing Dates: {len(future_dates)}")
    else:
        print("\n5. Billing Date column not found. Skipping future date check.")

    # Check for inconsistent departments
    department_column = 'Department' if 'Department' in df.columns else 'department' if 'department' in df.columns else None
    if department_column:
        unique_departments = df[department_column].unique()
        print("\n6. Unique Departments:", unique_departments.tolist())
    else:
        print("\n6. Department column not found. Skipping department check.")

    # Check for outliers in Amount
    if amount_column:
        Q1 = df[amount_column].quantile(0.25)
        Q3 = df[amount_column].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[amount_column] < (Q1 - 1.5 * IQR)) | (df[amount_column] > (Q3 + 1.5 * IQR))]
        print(f"\n7. Potential Outliers in Amount: {len(outliers)}")
    else:
        print("\n7. Amount column not found. Skipping outlier check.")

    return df

df_validated = validate_data(df)

Data Validation Report

1. Missing Values:
Series([], dtype: int64)

2. Duplicate Transactions: 0

3. Non-standard Statuses: 5
Unique non-standard statuses: ['Pending', 'Paid', 'Unpaid', 'Unknown']

4. Negative Amounts: 0

5. Future Billing Dates: 0

6. Unique Departments: ['Pediatrics', 'Orthopedics', 'Pharmacy', 'ENT', 'General Practice', 'Neurology', 'Cardiology', 'Dermatology', 'Unknown', 'Radiology', 'Oncology']

7. Potential Outliers in Amount: 12


**Billing Analysis**

In [3]:
def analyze_billing(df):
    print("Billing Analysis Report")
    print("======================")

    # Basic statistics
    print("\n1. Basic Statistics:")
    print(df['Amount'].describe())

    # Payment by status
    payment_by_status = df.groupby('Status')['Amount'].agg(['sum', 'mean', 'count'])
    print("\n2. Payment by Status:")
    print(payment_by_status)

    # Payment by department
    payment_by_dept = df.groupby('Department')['Amount'].agg(['sum', 'mean', 'count'])
    print("\n3. Payment by Department:")
    print(payment_by_dept)

    # Time series analysis
    df['Billing Date'] = pd.to_datetime(df['Billing Date'])
    monthly_totals = df.set_index('Billing Date').resample('M')['Amount'].sum()
    print("\n4. Monthly Billing Totals:")
    print(monthly_totals)

    # Identify high-cost transactions (e.g., top 5%)
    high_cost_threshold = df['Amount'].quantile(0.95)
    high_cost_transactions = df[df['Amount'] > high_cost_threshold]
    print(f"\n5. High-cost Transactions (top 5%): {len(high_cost_transactions)}")

    return df

df_analyzed = analyze_billing(df_validated)

Billing Analysis Report

1. Basic Statistics:
count     110.000000
mean      479.309018
std       738.759966
min        50.581003
25%       145.411712
50%       253.709053
75%       392.219435
max      3849.771331
Name: Amount, dtype: float64

2. Payment by Status:
                  sum        mean  count
Status                                  
Paid     12025.309679  316.455518     38
Pending  29588.746892  896.628694     33
Unknown    996.585426  199.317085      5
Unpaid   10113.349995  297.451470     34

3. Payment by Department:
                           sum        mean  count
Department                                       
Cardiology         3435.351394  229.023426     15
Dermatology        2957.750657  295.775066     10
ENT               12327.746765  880.553340     14
General Practice   4174.839297  596.405614      7
Neurology          4101.849231  455.761026      9
Oncology           8110.286370  675.857197     12
Orthopedics        1643.767133  234.823876      7
Pediatrics 

**Data Visualization**

In [4]:
def create_visualizations(df):
    # Distribution of Billing Amounts by Department
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='Department', y='Amount', data=df)
    plt.title('Distribution of Billing Amounts by Department')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('billing_by_department.png')
    plt.close()

    # Distribution of Transaction Statuses
    plt.figure(figsize=(10, 6))
    df['Status'].value_counts().plot(kind='pie', autopct='%1.1f%%')
    plt.title('Distribution of Transaction Statuses')
    plt.savefig('transaction_status_distribution.png')
    plt.close()

    # Monthly Billing Totals
    monthly_totals = df.set_index('Billing Date').resample('M')['Amount'].sum()
    plt.figure(figsize=(12, 6))
    monthly_totals.plot()
    plt.title('Monthly Billing Totals')
    plt.xlabel('Date')
    plt.ylabel('Total Amount')
    plt.savefig('monthly_billing_totals.png')
    plt.close()

create_visualizations(df_analyzed)

**Report Generation**

In [5]:
def generate_report(df):
    report = "Comprehensive Billing Audit Report\n"
    report += "==================================\n\n"

    # Data Overview
    report += "1. Data Overview:\n"
    report += f"   - Total transactions: {len(df)}\n"
    report += f"   - Date range: {df['Billing Date'].min()} to {df['Billing Date'].max()}\n"
    report += f"   - Total billing amount: ${df['Amount'].sum():.2f}\n\n"

    # Data Quality
    report += "2. Data Quality Issues:\n"
    report += f"   - Missing values: {df.isnull().sum().sum()}\n"
    report += f"   - Duplicate transactions: {len(df[df.duplicated()])}\n"
    report += f"   - Non-standard statuses: {len(df[~df['Status'].isin(['Paid', 'Unpaid', 'Pending'])])}\n"
    report += f"   - Negative amounts: {len(df[df['Amount'] < 0])}\n"
    report += f"   - Future billing dates: {len(df[df['Billing Date'] > datetime.now()])}\n\n"

    # Billing Analysis
    report += "3. Billing Analysis:\n"
    report += f"   - Average payment amount: ${df['Amount'].mean():.2f}\n"
    report += "   - Payment by status:\n"
    report += df.groupby('Status')['Amount'].sum().to_string() + "\n\n"
    report += "   - Payment by department:\n"
    report += df.groupby('Department')['Amount'].sum().to_string() + "\n\n"
    report += f"   - High-cost transactions (top 5%): {len(df[df['Amount'] > df['Amount'].quantile(0.95)])}\n"
    
    return report

final_report = generate_report(df_analyzed)

# Save the report to a file
with open('billing_audit_report.txt', 'w') as f:
    f.write(final_report)

print("Report saved to 'billing_audit_report.txt'")
print(final_report)

Report saved to 'billing_audit_report.txt'
Comprehensive Billing Audit Report

1. Data Overview:
   - Total transactions: 110
   - Date range: 2023-01-01 00:00:00 to 2023-04-10 00:00:00
   - Total billing amount: $52723.99

2. Data Quality Issues:
   - Missing values: 0
   - Duplicate transactions: 0
   - Non-standard statuses: 5
   - Negative amounts: 0
   - Future billing dates: 0

3. Billing Analysis:
   - Average payment amount: $479.31
   - Payment by status:
Status
Paid       12025.309679
Pending    29588.746892
Unknown      996.585426
Unpaid     10113.349995

   - Payment by department:
Department
Cardiology           3435.351394
Dermatology          2957.750657
ENT                 12327.746765
General Practice     4174.839297
Neurology            4101.849231
Oncology             8110.286370
Orthopedics          1643.767133
Pediatrics           5561.996201
Pharmacy             3752.991825
Radiology            1657.427957
Unknown              4999.985161

   - High-cost transacti